In [4]:
import numpy as np
import pandas as pd

In [8]:
# split to hour, minute, second

def add_time_string(df):
    df = df.reset_index(drop=True)
    df["queryts_new"] = pd.DataFrame(pd.to_datetime(df.queryts, unit='s'))
    df.queryts_new = df.queryts_new.astype(str)

    con = pd.DataFrame(df.queryts_new.str.split(" ",2).tolist(), columns = ['a','b'])
    con1 = pd.DataFrame(con.b.str.split(":",3).tolist(), columns = ['hr','min','sec'])
    df["hr"] = con1.hr
    df.hr = pd.to_numeric(df.hr, errors = 'coerce')
    
    df = df.drop(
        ["customer_id","queryts","product_id","month","date","date_new","queryts_new"], axis=1)

    print (df.shape)
    return df

In [14]:
def count_interval_six(x):
    x = pd.Series(x)
    hr5 = sum(x < 6)
    hr11 = sum(x < 12)
    hr17 = sum(x < 18)
    hr23 = sum(x < 24)
    hr_0_5 = hr5
    hr_6_11 = hr11 - hr5
    hr_12_17 = hr17 - hr11
    hr_18_23 = hr23 - hr17
    return hr_0_5, hr_6_11, hr_12_17, hr_18_23

def groupby_count_interval_six(df):
    f = {'hr': lambda x: count_interval_six(x)}

    g = df.groupby(["file_id"]).aggregate(f).reset_index()
    g.columns = ["file_id", "count_interval_six_list"]

    g_ = g['count_interval_six_list'].apply(pd.Series)
    g_.columns = ["hr_0_5", "hr_6_11", "hr_12_17", "hr_18_23"]
    g = pd.concat([g[["file_id"]], g_], axis=1)
    
    return g

In [15]:
df = pd.DataFrame()
for file_ in ["train1","train2","train3","valid"]:
    df_ = pd.read_pickle("dataset/" + str(file_) + ".pkl")
    df = pd.concat([df, df_], axis=0)
    
df = add_time_string(df)
g_train = groupby_count_interval_six(df)

print (g_train.shape)

(54250245, 2)
(52518, 5)


In [16]:
df = pd.DataFrame()
for file_ in ["test_1","test_2","test_3"]:
    df_ = pd.read_pickle("dataset/" + str(file_) + ".pkl")
    df = pd.concat([df, df_], axis=0)

df = add_time_string(df)
g_test = groupby_count_interval_six(df)

print (g_test.shape)

(29022865, 2)
(29376, 5)


In [20]:
g = pd.concat([g_train, g_test], axis=0).reset_index(drop=True)

In [21]:
g.describe()

,hr_0_5,hr_6_11,hr_12_17,hr_18_23
count,81894.000000,81894.000000,81894.000000,81894.000000
mean,254.510550,254.407588,258.051970,249.870076
std,680.375733,639.861477,669.656205,723.096444
min,0.000000,0.000000,0.000000,0.000000
25%,15.000000,20.000000,13.000000,5.000000
50%,63.000000,72.000000,61.000000,34.000000
75%,202.000000,225.000000,212.000000,160.000000
max,35320.000000,50739.000000,25452.000000,20588.000000


In [22]:
g.to_csv("dataset/count_interval_six.csv", index=False)